# Python Lectures (2)

Last time we did a brief introduction. We were writing a small application that calculates currency exchange. This time we are going to extend the application by pulling the exchange rates from the web.  For that I am going to need two libraries:

In [1]:
import urllib
import xmltodict

The first library `urllib` is for pulling content from the web, while the second `xmltodict` for parsing an XML stream into a python dictionary (map).

In [3]:
raw = xmltodict.parse(urllib.request.urlopen("http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"))

[The European Central Bank](http://www.ecb.europa.eu/) keeps daily data on exchange rate at the URL you see above.  The data is kept in [XML](http://en.wikipedia.org/wiki/XML) format.  When we call the function `urllib.urlopen` on the URL, we get a python stream: something like a file.  Then the function `xmltodict.parse` parses the content of the stream into a python map.  The contents of the map complicated:

In [4]:
raw

OrderedDict([('gesmes:Envelope',
              OrderedDict([('@xmlns:gesmes',
                            'http://www.gesmes.org/xml/2002-08-01'),
                           ('@xmlns',
                            'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'),
                           ('gesmes:subject', 'Reference rates'),
                           ('gesmes:Sender',
                            OrderedDict([('gesmes:name',
                                          'European Central Bank')])),
                           ('Cube',
                            OrderedDict([('Cube',
                                          OrderedDict([('@time', '2017-05-16'),
                                                       ('Cube',
                                                        [OrderedDict([('@currency',
                                                                       'USD'),
                                                                      ('@rate',
                    

Everything in python is an object, and objects have admits interfaces. Maps have two fundamental interfaces: keys and values.

In [5]:
Rates = {"US": 1.0, "EUR": 0.8, "TRY": 2.24, "BITCOIN": 0.0026}

print(Rates.keys())
print(Rates.values())

dict_keys(['EUR', 'BITCOIN', 'US', 'TRY'])
dict_values([0.8, 0.0026, 1.0, 2.24])


So, let us see what the keys are in the raw data we pulled of from the European Central Bank:

In [6]:
raw.keys()

odict_keys(['gesmes:Envelope'])

This is funny. There is only one key. If you do data analysis long enough, you'll see that XML, or JSON, data you pulled is structured as smartly (or stupidly) as the people who design them.  Let us move in recursively:

In [7]:
raw[u'gesmes:Envelope']

OrderedDict([('@xmlns:gesmes', 'http://www.gesmes.org/xml/2002-08-01'),
             ('@xmlns', 'http://www.ecb.int/vocabulary/2002-08-01/eurofxref'),
             ('gesmes:subject', 'Reference rates'),
             ('gesmes:Sender',
              OrderedDict([('gesmes:name', 'European Central Bank')])),
             ('Cube',
              OrderedDict([('Cube',
                            OrderedDict([('@time', '2017-05-16'),
                                         ('Cube',
                                          [OrderedDict([('@currency', 'USD'),
                                                        ('@rate', '1.1059')]),
                                           OrderedDict([('@currency', 'JPY'),
                                                        ('@rate', '125.67')]),
                                           OrderedDict([('@currency', 'BGN'),
                                                        ('@rate', '1.9558')]),
                                           Ordere

Another map/dictionary! Hmm... The keys?

In [8]:
raw[u'gesmes:Envelope'].keys()

odict_keys(['@xmlns:gesmes', '@xmlns', 'gesmes:subject', 'gesmes:Sender', 'Cube'])

In [9]:
raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']

[OrderedDict([('@currency', 'USD'), ('@rate', '1.1059')]),
 OrderedDict([('@currency', 'JPY'), ('@rate', '125.67')]),
 OrderedDict([('@currency', 'BGN'), ('@rate', '1.9558')]),
 OrderedDict([('@currency', 'CZK'), ('@rate', '26.420')]),
 OrderedDict([('@currency', 'DKK'), ('@rate', '7.4394')]),
 OrderedDict([('@currency', 'GBP'), ('@rate', '0.85868')]),
 OrderedDict([('@currency', 'HUF'), ('@rate', '309.54')]),
 OrderedDict([('@currency', 'PLN'), ('@rate', '4.1984')]),
 OrderedDict([('@currency', 'RON'), ('@rate', '4.5478')]),
 OrderedDict([('@currency', 'SEK'), ('@rate', '9.7215')]),
 OrderedDict([('@currency', 'CHF'), ('@rate', '1.0958')]),
 OrderedDict([('@currency', 'NOK'), ('@rate', '9.3918')]),
 OrderedDict([('@currency', 'HRK'), ('@rate', '7.4303')]),
 OrderedDict([('@currency', 'RUB'), ('@rate', '62.3000')]),
 OrderedDict([('@currency', 'TRY'), ('@rate', '3.9330')]),
 OrderedDict([('@currency', 'AUD'), ('@rate', '1.4920')]),
 OrderedDict([('@currency', 'BRL'), ('@rate', '3.4316'

This is really stupid. The data is an array of small dictionaries each of size 1 containing the name of the currency and the rate.  We need to put the data in the correct format.

In [10]:
stupid = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']

Rates = { x['@currency']: float(x[u'@rate']) for x in stupid }

Above, I used a construction called *comprehension*. Let me explain: assume you want a range of integers for some reason.  In python you call

In [11]:
range(10)

range(0, 10)

In [12]:
range(2,12)

range(2, 12)

In [13]:
range(24,3,-2)

range(24, 3, -2)

But what if I want the squares of the even numbers between 2 and 26 stored in an array?

In [14]:
[x*x for x in range(2,26,2)]

[4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576]

How about stored in a set?

In [15]:
{x*x for x in range(2,26,2)}

{4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576}

How about a dictionary?

In [16]:
{ "Index-"+str(i): i*i for i in range(2,26,2) }

{'Index-10': 100,
 'Index-12': 144,
 'Index-14': 196,
 'Index-16': 256,
 'Index-18': 324,
 'Index-2': 4,
 'Index-20': 400,
 'Index-22': 484,
 'Index-24': 576,
 'Index-4': 16,
 'Index-6': 36,
 'Index-8': 64}

Let us check the contents of `stupid`:

In [17]:
stupid[1]

OrderedDict([('@currency', 'JPY'), ('@rate', '125.67')])

In [18]:
stupid[1][u'@currency']

'JPY'

In [19]:
stupid[1][u'@rate']

'125.67'

In [20]:
Rates = { x['@currency']: float(x[u'@rate']) for x in stupid }

In [21]:
Rates

{'AUD': 1.492,
 'BGN': 1.9558,
 'BRL': 3.4316,
 'CAD': 1.5058,
 'CHF': 1.0958,
 'CNY': 7.6185,
 'CZK': 26.42,
 'DKK': 7.4394,
 'GBP': 0.85868,
 'HKD': 8.6143,
 'HRK': 7.4303,
 'HUF': 309.54,
 'IDR': 14709.02,
 'ILS': 3.9874,
 'INR': 70.8505,
 'JPY': 125.67,
 'KRW': 1234.76,
 'MXN': 20.6886,
 'MYR': 4.7791,
 'NOK': 9.3918,
 'NZD': 1.609,
 'PHP': 54.898,
 'PLN': 4.1984,
 'RON': 4.5478,
 'RUB': 62.3,
 'SEK': 9.7215,
 'SGD': 1.5457,
 'THB': 38.187,
 'TRY': 3.933,
 'USD': 1.1059,
 'ZAR': 14.5287}

Notice that "EUR" is missing since this is a data from European Central Banks, the rates are all relative to "EUR". We add that extra:

In [22]:
Rates.update({u'EUR': 1.00})

In [23]:
Rates

{'AUD': 1.492,
 'BGN': 1.9558,
 'BRL': 3.4316,
 'CAD': 1.5058,
 'CHF': 1.0958,
 'CNY': 7.6185,
 'CZK': 26.42,
 'DKK': 7.4394,
 'EUR': 1.0,
 'GBP': 0.85868,
 'HKD': 8.6143,
 'HRK': 7.4303,
 'HUF': 309.54,
 'IDR': 14709.02,
 'ILS': 3.9874,
 'INR': 70.8505,
 'JPY': 125.67,
 'KRW': 1234.76,
 'MXN': 20.6886,
 'MYR': 4.7791,
 'NOK': 9.3918,
 'NZD': 1.609,
 'PHP': 54.898,
 'PLN': 4.1984,
 'RON': 4.5478,
 'RUB': 62.3,
 'SEK': 9.7215,
 'SGD': 1.5457,
 'THB': 38.187,
 'TRY': 3.933,
 'USD': 1.1059,
 'ZAR': 14.5287}

Now, let us recall our program:

In [24]:
def Exchange(amount, source, target):
    return(amount * Rates[target]/Rates[source])

print(Exchange(100.0, "TRY", u'EUR'))

25.425883549453346


Let me write a new function that updates the exchange rate dictionary:

In [25]:
def UpdateRates():
    raw = xmltodict.parse(urllib.urlopen("http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"))
    core = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']
    Rates = { x['@currency']: float(x[u'@rate']) for x in core }

## A Finished Version

I prefer to interact with my program on the command line.  So, I will everything together as follows:

    ## Back End

    import urllib
    import xmltodict

    URL = "http://www.ecb.europa.eu/stats/eurofxref/eurofxref-daily.xml"

    def DownloadRates(localFile):
      urllib.urlretrieve(URL, filename=localFile)

    def GetRates(localFile):
      raw = xmltodict.parse(open(localFile))
      core = raw[u'gesmes:Envelope'][u'Cube'][u'Cube'][u'Cube']
      Rates = { x['@currency']: float(x[u'@rate']) for x in core }
      Rates.update({u'EUR': 1.00})
      return(Rates)

    def Exchange(Rates,amount,source,target):
      return(amount * Rates[target]/Rates[source])


    ## Front End

    import sys

    FILE = "/home/kaygun/local/lib/rates.xml"

    if(sys.argv[1] == "refresh"):
      DownloadRates(FILE)
    elif(sys.argv[1] == "list"):
      Rates = GetRates(FILE)
      Currencies = Rates.keys()
      Currencies.sort()
      for x in Currencies:
        print x, Rates[x]
    else:
      Rates = GetRates(FILE)
      amount = float(sys.argv[1])
      source = sys.argv[2]
      target = sys.argv[3]
      print str(amount), source, "is", "{0:.2f}".format(Exchange(Rates,amount,source,target)), target
